# Encoder with Gaussian Relu
This Encoder is built with Gaussian Relu which is also the activation function used in GPT models.

In [1]:
import torch
import math
from torch import nn
import torch.nn.functional as F

### Positional Encoding
$$ PE_{(pos,2i)} = \sin\left(\frac{pos}{10000^{2i/d_{\text{model}}}}\right) $$

The Positional_Encoding class is a PyTorch module that performs positional encoding on the input tensor. It has three attributes defined in its constructor method: d_model, hidden, and drop_prob. These attributes specify the dimensionality of the input tensor, the size of the hidden layer in the linear layers, and the dropout probability, respectively.

The forward method of the class applies two linear layers to the input tensor, followed by the ReLU activation function and a dropout layer. The first linear layer reduces the dimensionality of the input tensor from d_model to hidden, while the second linear layer restores it back to d_model. The ReLU activation function introduces non-linearity into the network, and the dropout layer helps prevent overfitting by randomly dropping out units during training.

The output tensor of the forward method has the same dimensionality as the input tensor, but with the positional encoding applied to it. Overall, the class provides a way to perform positional encoding on input tensors using linear layers, activation functions, and dropout regularization.

In [2]:
class Positional_Encoding(nn.Module):
    #------------------constructor-----------------#
    def __init__(self,d_model,hidden,drop_prob=0.1):
        super(Positional_Encoding,self).__init__()
        self.linear1 = nn.linear(d_model,hidden)            #----First linear layer
        self.linear2 = nn.Linear(hidden, d_model)           #----Second linear layer
        self.relu = nn.ReLU()                               #----Activation function
        self.dropout = nn.Dropout(p=drop_prob)              #----Layer defining dropout
    #------------------forward pass-----------------#
    def forward(self,x):
        x = self.linear1(x)
        print(f" Input x after first linear layer: {x.size()}")
        x = self.relu(x)
        print(f"x after activation: {x.size()}")
        x = self.dropout(x)
        print(f"x after dropout: {x.size()}")
        x = self.linear2(x)
        print(f"x after 2nd linear layer: {x.size()}")
        return x

### The Self Attention Equation is given

$$Self-Attention = softmax(\frac{Q.K^T}{\sqrt(d_k)}+M)V$$


The `Q.K^T` operation can be thought of as a similarity measure between the query and key vectors. The dot product of two vectors measures the **cosine similarity** between them, which ranges from -1 (opposite directions) to 1 (same direction). 

The $\sqrt d_k$ is included in the equation to reduce the variance of the cosine similarity calculated above.

In [3]:
def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1]
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k)
    print(f"scaled.size() : {scaled.size()}")
    if mask is not None:
        print(f"-- ADDING MASK of shape {mask.size()} --") 
        # Broadcasting add. So just the last N dimensions need to match
        scaled += mask
    attention = F.softmax(scaled, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention

class MultiHeadAttention(nn.Module):

    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.qkv_layer = nn.Linear(d_model , 3 * d_model)
        self.linear_layer = nn.Linear(d_model, d_model)
    
    def forward(self, x, mask=None):
        batch_size, max_sequence_length, d_model = x.size()
        print(f"x.size(): {x.size()}")
        qkv = self.qkv_layer(x)
        print(f"qkv.size(): {qkv.size()}")
        qkv = qkv.reshape(batch_size, max_sequence_length, self.num_heads, 3 * self.head_dim)
        print(f"qkv.size(): {qkv.size()}")
        qkv = qkv.permute(0, 2, 1, 3)
        print(f"qkv.size(): {qkv.size()}")
        q, k, v = qkv.chunk(3, dim=-1)
        print(f"q size: {q.size()}, k size: {k.size()}, v size: {v.size()}, ")
        values, attention = scaled_dot_product(q, k, v, mask)
        print(f"values.size(): {values.size()}, attention.size:{ attention.size()} ")
        values = values.reshape(batch_size, max_sequence_length, self.num_heads * self.head_dim)
        print(f"values.size(): {values.size()}")
        out = self.linear_layer(values)
        print(f"out.size(): {out.size()}")
        return out

### LayerNormalization
- B: The batch size
- S: Number of input words 
- E: Input word embedding size

The Layer Normalization has 2 learnable parameters which are $\gamma$ & $\beta$ . We will calculate them as the model trains , but for initializing purpose we take $\gamma$ = matrix of ones & $\beta$ = matrix of zeros. They are updated in backpropagation using gradient descent steps.

In [4]:
class LayerNormalization(nn.Module):
    def __init__(self, parameters_shape, eps=1e-5):
        super().__init__()
        self.parameters_shape=parameters_shape
        self.eps=eps
        self.gamma = nn.Parameter(torch.ones(parameters_shape))
        self.beta =  nn.Parameter(torch.zeros(parameters_shape))

    def forward(self, inputs):
        dims = [-(i + 1) for i in range(len(self.parameters_shape))]
        mean = inputs.mean(dim=dims, keepdim=True)
        print(f"Mean ({mean.size()})")
        var = ((inputs - mean) ** 2).mean(dim=dims, keepdim=True)
        std = (var + self.eps).sqrt()
        print(f"Standard Deviation  ({std.size()})")
        y = (inputs - mean) / std
        print(f"y: {y.size()}")
        out = self.gamma * y  + self.beta
        print(f"self.gamma: {self.gamma.size()}, self.beta: {self.beta.size()}")
        print(f"out: {out.size()}")
        return out

### Feed Forward Neural Net

In [5]:
class GRELU(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        # Compute the Gaussian CDF of the input tensor
        cdf = 0.5 * (1.0 + torch.erf(x / torch.sqrt(torch.tensor(2.0))))

        # Compute the GRELU function element-wise
        output = x * cdf

        return output

In [6]:
class PositionwiseFeedForward(nn.Module):

    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden)
        self.linear2 = nn.Linear(hidden, d_model)
        self.glue= GRELU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
        x = self.linear1(x)
        print(f"x after first linear layer: {x.size()}")
        x = self.glue(x)
        print(f"x after activation: {x.size()}")
        x = self.dropout(x)
        print(f"x after dropout: {x.size()}")
        x = self.linear2(x)
        print(f"x after 2nd linear layer: {x.size()}")
        return x


### Encoder Block
Encoder block represents a single block of encoder.

In [7]:
class EncoderBlock(nn.Module):

    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(EncoderBlock, self).__init__()
        self.attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)
        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)
    
    def forward(self, x):
        residual_x = x
        print("------- ATTENTION 1 ------")
        x = self.attention(x, mask=None)
        print("------- DROPOUT 1 ------")
        x = self.dropout1(x)
        print("------- ADD AND LAYER NORMALIZATION 1 ------")
        x = self.norm1(x + residual_x)
        residual_x = x
        print("------- ATTENTION 2 ------")
        x = self.ffn(x)
        print("------- DROPOUT 2 ------")
        x = self.dropout2(x)
        print("------- ADD AND LAYER NORMALIZATION 2 ------")
        x = self.norm2(x + residual_x)
        return x

### Encoder 
Encoder represents the entire Encoder Architecture.

In [8]:
class Encoder(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob, num_layers):
        super().__init__()
        self.layers = nn.Sequential(*[EncoderBlock(d_model, ffn_hidden, num_heads, drop_prob)
                                     for _ in range(num_layers)])

    def forward(self, x):
        x = self.layers(x)
        return x

### Exmaple

In [9]:
#--------------Defining the parameters-------------#
d_model = 512
num_heads = 8
drop_prob = 0.1
batch_size = 30
max_sequence_length = 200
ffn_hidden = 2048
num_layers = 5

In [10]:
#--------------Building the Encoder-------------#
encoder = Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers)

In [12]:
x = torch.randn( (batch_size, max_sequence_length, d_model) ) # includes positional encoding
out = encoder(x)

------- ATTENTION 1 ------
x.size(): torch.Size([30, 200, 512])
qkv.size(): torch.Size([30, 200, 1536])
qkv.size(): torch.Size([30, 200, 8, 192])
qkv.size(): torch.Size([30, 8, 200, 192])
q size: torch.Size([30, 8, 200, 64]), k size: torch.Size([30, 8, 200, 64]), v size: torch.Size([30, 8, 200, 64]), 
scaled.size() : torch.Size([30, 8, 200, 200])
values.size(): torch.Size([30, 8, 200, 64]), attention.size:torch.Size([30, 8, 200, 200]) 
values.size(): torch.Size([30, 200, 512])
out.size(): torch.Size([30, 200, 512])
------- DROPOUT 1 ------
------- ADD AND LAYER NORMALIZATION 1 ------
Mean (torch.Size([30, 200, 1]))
Standard Deviation  (torch.Size([30, 200, 1]))
y: torch.Size([30, 200, 512])
self.gamma: torch.Size([512]), self.beta: torch.Size([512])
out: torch.Size([30, 200, 512])
------- ATTENTION 2 ------
x after first linear layer: torch.Size([30, 200, 2048])
x after activation: torch.Size([30, 200, 2048])
x after dropout: torch.Size([30, 200, 2048])
x after 2nd linear layer: torch.